# H(div)-conforming Stokes


$H(div)$-conforming hybrid DG [Lehrenfeld+Schöberl, 2016]

Find $u \in BDM^k, \hat u \in P^k_{t}, p \in P^k$ ...

In [1]:
import netgen.gui
from netgen.geom2d import SplineGeometry
from ngsolve import *

In [2]:
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.08))
mesh.Curve(5)

In [3]:
order=4
VT = HDiv(mesh, order=order, dirichlet="wall|cyl|inlet")
VF = VectorFacet(mesh, order=order, dirichlet="wall|cyl|inlet")
Q = L2(mesh, order=order-1)
V = FESpace ([VT,VF,Q])

u, uhat, p = V.TrialFunction()
v, vhat, q = V.TestFunction()

In [4]:
n = specialcf.normal(mesh.dim)
h = specialcf.mesh_size

def tang(vec):
    return vec - (vec*n)*n

# Thesis Christoph Lehrenfeld, page 71
a = BilinearForm (V)
nu = 1e-3
a += SymbolicBFI (nu*InnerProduct(grad(u), grad(v)))
a += SymbolicBFI (nu*InnerProduct(grad(u).trans*n, tang(vhat-v)), element_boundary=True )
a += SymbolicBFI (nu*InnerProduct(grad(v).trans*n, tang(uhat-u)), element_boundary=True )
a += SymbolicBFI (nu*4*order*order/h * InnerProduct(tang(vhat-v), tang(uhat-u)), element_boundary=True )
a += SymbolicBFI (div(u)*q + div(v)*p)
a.Assemble()

In [6]:
f = LinearForm(V)
f.Assemble()

invstokes = a.mat.Inverse(V.FreeDofs())

gfu = GridFunction(V)
uin = CoefficientFunction((1.5*4*y*(0.41-y)/(0.41*0.41),0))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

In [7]:
res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += invstokes * res
Draw (gfu.components[0], mesh, "vel") # velocity
Draw (gfu.components[2], mesh, "pressure") # pressure